<a href="https://colab.research.google.com/github/aritejhg/tinyyolov4_person_detector/blob/main/Door_detector_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#collins people at door detector

Objectives:
1. Convert Sunmi and Top View Multiple Person dataset to YOLO format (done)
2. Implement histogram equalisation
3. Get baseline from KDE background subtraction + Historgram
4. Train TinyYOLOv4 model and check performance. Implement with DIoU + Multi-Scale
5. If TinyYOLOV4 is better, convert it to tf-lite
Result required: Just a indication if people are present in the area. To be passed to other code for talking to MiR
6. compare against efficientdet-lite?

Max resolution of feed?

Return the highest confidence level of human detection?

Need to return yes/no for occupancy, 90-95% accurate


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


References:
1. [Training tinyYOLOv4](https://colab.research.google.com/drive/1hQO4nOoD6RDxdbz3C1YSiifTsyZjZpYm?usp=sharing)
2. [Top View Multi Person labelled images zip](https://drive.google.com/drive/folders/1dc1QrDvZPWXbUfzEQE5EPIAz2CPCVI38)
3. [Sunmi dataset](https://github.com/Sunmi-AI-Lab/head-detection-and-tracking/tree/master/datasets) 
4. [DIoU](https://arxiv.org/abs/1911.08287)
5. [Multi-Scale tinyYOLOv4 Research paper](https://link.springer.com/chapter/10.1007/978-3-030-85383-9_1)
6. [convert to tflite](https://github.com/DoranLyong/yolov4-tiny-tflite-for-person-detection)

In [ ]:
#imports 
import os
import shutil
import random
import glob
import fnmatch

# Datasets

## Download datasets

In [ ]:
#imports
!pip install gdown
import gdown

###Sunmi dataset

In [ ]:
!apt install subversion

In [ ]:
!svn checkout https://github.com/Sunmi-AI-Lab/head-detection-and-tracking/trunk/datasets/in-office /content/Sunmi/


### Top View 


In [ ]:
os.makedirs("/content/TVMP")

In [ ]:
#https://drive.google.com/drive/folders/1dc1QrDvZPWXbUfzEQE5EPIAz2CPCVI38?usp=sharing
!gdown --folder https://drive.google.com/drive/folders/1dc1QrDvZPWXbUfzEQE5EPIAz2CPCVI38?usp=sharing /content/TVMP/

In [ ]:
!unzip /content/TVMP/labeled_images/test.zip

In [ ]:
!unzip /content/TVMP/labeled_images/train.zip 

##Backup downloaded datasets to drive

In [ ]:
shutil.copytree("/content/Sunmi","/content/drive/MyDrive/dataset/")

In [ ]:
shutil.copytree("/content/TVMP","/content/drive/MyDrive/dataset/TVMP")

##Converting datasets to YOLO format


While TVMP appears to be in YOLO format, Sunmi is in VOC XML with xmin ymin xmax ymax.

YOLO format is x_center, y_center, width, height. 

Conversion is required for sunmi using script below


Some files (hide annotations under Sunmi) have a different xml format, where they use xmin xmax ymin ymax instead of xmin ymin xmax ymax. These weird files also do not have width and height data at the start, so we use that to separate out the processing so that info is passed in the correct order to xml_to_yolo_bbox function.


In [ ]:
# inspired from https://towardsdatascience.com/convert-pascal-voc-xml-to-yolo-for-object-detection-f969811ccba5
import glob
import xml.etree.ElementTree as ET


classes = ["head"]
input_dir = "/content/drive/MyDrive/dataset/in-office/annotations/"
output_dir = "/content/drive/MyDrive/dataset/in-office/labels/"
image_dir = "/content/drive/MyDrive/dataset/in-office/images/"

def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h
    return [x_center, y_center, width, height]

files = glob.glob(os.path.join(input_dir, '*.xml'))
for i,fil in enumerate(files):
    #print (fil)
    basename = os.path.basename(fil)
    filename = os.path.splitext(basename)[0]
    if not os.path.exists(os.path.join(image_dir, f"{filename}.jpg")):
        print(f"{filename} image does not exist!")
        continue
    
    result = []
    os.makedirs("/content/drive/MyDrive/dataset/in-office/labels", exist_ok = True)
    # parse the content of the xml file
    tree = ET.parse(fil)
    root = tree.getroot()
    try: 
      width = int(root.find("size").find("width").text)
      height = int(root.find("size").find("height").text)
      for obj in root.findall('object'):
        # since we are using only 1 class, this isnt necessary
        # label = obj.find("name").text
        # # check for new classes and append to list
        # if label not in classes:
        #     classes.append(label)
        # index = classes.index(label)
        pil_bbox = [int(x.text) for x in obj.find("bndbox")]
        yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)
        
    except: 
      width, height = 800, 800
      print(fil)
      for obj in root.findall('object'):
        # since we are using only 1 class, this isnt necessary
        # label = obj.find("name").text
        # # check for new classes and append to list
        # if label not in classes:
        #     classes.append(label)
        # index = classes.index(label)
        pil_bbox = [int(x.text) for x in obj.find("bndbox")]
        final_box = [pil_bbox[0],pil_bbox[2],pil_bbox[1],pil_bbox[3]] #done coz these files have a xmin xmax ymin ymax format
        yolo_bbox = xml_to_yolo_bbox(final_box, width, height)

    # convert data to string
    bbox_string = " ".join([str(x) for x in yolo_bbox])
    result.append(f"0 {bbox_string}")
    if result:
        # generate a YOLO format text file for each xml file
        with open(os.path.join(output_dir, f"{filename}.txt"), "w", encoding="utf-8") as f:
            f.write("\n".join(result))
        print (f"{i+1}/{len(files)} done")
# the ones with no height width data are giving a negative bbox value when converted to yolo, needs to be checked

##Combine & create train/val/test folders

80-10-10 split

In [ ]:
# copy over both datasets to a new folder
for folder in ["train",'test', 'val', 'raw']:
  os.makedirs("/content/dataset/"+folder, exist_ok=True)

for files in os.listdir("/content/drive/MyDrive/dataset/in-office/images"):
  print(files)
  shutil.copy("/content/drive/MyDrive/dataset/in-office/images/"+files, "/content/dataset/raw/")
!cp -r /content/drive/MyDrive/dataset/TVMP/labeled_images/train/. /content/dataset/raw # j another method of copying
!cp -r /content/drive/MyDrive/dataset/TVMP/labeled_images/test/. /content/dataset/raw

!rm -rf -d /content/dataset/raw/*.txt
# coz TVMP has images and txt tgt so we can just delete first, divide up the images only into train test val, then load up annotations

In [ ]:
#sanity check to confirm total number of images
print(len(fnmatch.filter(os.listdir("/content/dataset/raw"), '*.jpg')))
print (len(fnmatch.filter(os.listdir("/content/drive/MyDrive/dataset/TVMP/labeled_images/test/"), '*.jpg'))+len(fnmatch.filter(os.listdir("/content/drive/MyDrive/dataset/TVMP/labeled_images/train/"), '*.jpg'))+len(fnmatch.filter(os.listdir("/content/drive/MyDrive/dataset/in-office/images"), '*.jpg')))

6075
6075


In [ ]:
random.seed(100) #seed so that rerunning will not change data in the folders
root_path = "/content/dataset/raw/"
# create list of images
list_of_images = list(set(fnmatch.filter(os.listdir("/content/dataset/raw"), '*.jpg')))
total_images = len(list_of_images)
# print(list_of_images)
# print(total_images)
# choose images for training, testing, and validation sets
train_images = []
val_images = []
test_images = []
for i in range(int(total_images*0.8)):
   train_choice = random.choice(list_of_images)
   train_images.append(train_choice)
   list_of_images.remove(train_choice)
for i in range(int(total_images*0.1)):
   val_choice = random.choice(list_of_images)
   val_images.append(val_choice)
   list_of_images.remove(val_choice)
test_images = list_of_images #since only 10% test data is left

print(len(train_images),len(val_images), len(test_images))
# print(test_images)

4860 607 608
['cam_5_2_and_helf_mins_00000269.jpg', 'jacket_1208.jpg', 'cam_1_2_and_helf_mins_00000091.jpg', 'cam6_1hour_00000124.jpg', 'cam_3_2_and_helf_mins_00000224.jpg', 'cam_6_2_and_helf_mins_00000251.jpg', 'cam1_1hour_00000089.jpg', 'cam_6_train_00000075.jpg', 'cam_2_trimmed_00000120.jpg', 'cam_1_2_and_helf_mins_00000113.jpg', 'cam_3_00000050.jpg', 'hide_0201.jpg', 'cam_6_train_00000100.jpg', 'cam_3_2_and_helf_mins_00000111.jpg', 'hide_0590.jpg', 'cam_3_trimmed_00000066.jpg', 'hide_0122.jpg', 'hide_0132.jpg', 'hide_0174.jpg', 'cam3_1hour_00000122.jpg', 'cam_2_00000045.jpg', 'cam_2_00000028.jpg', 'close_0359.jpg', 'close_0857.jpg', 'lightoff_0752.jpg', 'cam_1_00000094.jpg', 'hide_0107.jpg', 'cam_5_train_00000055.jpg', 'cam_5_train_00000095.jpg', 'cam_2_00000005.jpg', 'cam_1_2_and_helf_mins_00000085.jpg', 'cam1_1hour_00000091.jpg', 'hide_0099.jpg', 'cam_6_2_and_helf_mins_00000038.jpg', 'hide_0364.jpg', 'cam_2_2_and_helf_mins_00000088.jpg', 'cam_3_trimmed_00000017.jpg', 'cam1_1hour_

In [ ]:
#copy images based on the lists created 
root_path = "/content/dataset/"

for train_img in train_images:
  shutil.copy(root_path + "raw/" + train_img, os.path.join(root_path, "train/"))
for val_img in val_images:
  shutil.copy(root_path + "raw/" + val_img, os.path.join(root_path, "val/"))  
for test_img in test_images:
  shutil.copy(root_path + "raw/" + test_img, os.path.join(root_path, "test/"))  

In [ ]:
#sanity check, check numbers in each folder match up with print in cell earlier
print(len(fnmatch.filter(os.listdir("/content/dataset/train"), '*.jpg')))
print(len(fnmatch.filter(os.listdir("/content/dataset/val"), '*.jpg')))
print(len(fnmatch.filter(os.listdir("/content/dataset/test"), '*.jpg')))

4860
607
608


In [ ]:
#creating a raw_labels folder to find labels and copy into train/val/test folder. Inefficient overall but a 1-time run + easier to code 
os.makedirs("/content/dataset/raw_labels", exist_ok=True)
for files in  fnmatch.filter(os.listdir("/content/drive/MyDrive/dataset/in-office/labels"), '*.txt'):
  shutil.copy("/content/drive/MyDrive/dataset/in-office/labels/"+ files, "/content/dataset/raw_labels/")
for files in fnmatch.filter(os.listdir("/content/drive/MyDrive/dataset/TVMP/labeled_images/test/"), '*.txt'):
  shutil.copy("/content/drive/MyDrive/dataset/TVMP/labeled_images/test/"+ files, "/content/dataset/raw_labels/")
for files in fnmatch.filter(os.listdir("/content/drive/MyDrive/dataset/TVMP/labeled_images/train/"), '*.txt'):
  shutil.copy("/content/drive/MyDrive/dataset/TVMP/labeled_images/train/"+ files, "/content/dataset/raw_labels/")


In [ ]:
#sanity check. total files in raw labels
print(len(fnmatch.filter(os.listdir("/content/dataset/raw_labels"), '*.txt')))

6075


In [ ]:
#now to actl copy over labels
root_path = "/content/dataset/"

for train_img in train_images:
  shutil.copy(root_path + "raw_labels/" + train_img.rstrip(".jpg") + ".txt", os.path.join(root_path, "train/"))
for val_img in val_images:
  shutil.copy(root_path + "raw_labels/" + val_img.rstrip(".jpg") + ".txt", os.path.join(root_path, "val/"))  
for test_img in test_images:
  shutil.copy(root_path + "raw_labels/" + test_img.rstrip(".jpg") + ".txt", os.path.join(root_path, "test/"))  

In [ ]:
#sanity check, check numbers in each folder match up with print in cell earlier
print(len(fnmatch.filter(os.listdir("/content/dataset/train"), '*.txt')))
print(len(fnmatch.filter(os.listdir("/content/dataset/val"), '*.txt')))
print(len(fnmatch.filter(os.listdir("/content/dataset/test"), '*.txt')))

4860
607
608


In [ ]:
# saving final created dataset to gdrive
!cp -r /content/dataset/train /content/drive/MyDrive/dataset/final_dataset
!cp -r /content/dataset/val /content/drive/MyDrive/dataset/final_dataset
!cp -r /content/dataset/test /content/drive/MyDrive/dataset/final_dataset

# Data-preprocessing

Things needed:
1. Create torch.dataset thingy
2. resolution? currently j gon use 416*416
3. Preprocesses to be done?
4. Histogram equalization before feeding in the image?
5. transfer learning?


In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torch.utils.data import TensorDataset
# from torch.utils.data import DataLoader
# import torchvision
# from torchvision import datasets, models, transforms
# import time
# import os
# import random

In [ ]:
# def data_loader(batch_size = 32, mode = "train", root_path = "/content/"):
#   transform = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
# #experiment with grayscale?
#   if mode == 'train':
#     dataset = torchvision.datasets.ImageFolder(root_path + "train/", transform=transform)
#     dataloader = DataLoader(dataset, batch_size=batch_size, num_workers = 2, shuffle=True)
#   if mode == 'val':
#     dataset = torchvision.datasets.ImageFolder(root_path + "val/", transform=transform)
#     dataloader = DataLoader(dataset, batch_size=batch_size, num_workers = 2, shuffle=True)
#   if mode == 'test':
#     dataset = torchvision.datasets.ImageFolder(root_path + "test/", transform=transform)     
#     dataloader = DataLoader(dataset, batch_size=batch_size, num_workers = 2, shuffle=True)
#   return dataloader

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
!sudo apt install libopencv-dev
!make

In [ ]:
!cp /content/darknet/cfg/yolov4-tiny-custom.cfg /content/yolov4-tiny/

In [ ]:
# change makefile to have GPU and OPENCV enabled
# also set CUDNN, CUDNN_HALF and LIBSO to 1

%cd /content/darknet/
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

/content/darknet


In [77]:
!cd /content/darknet/data 
!find -maxdepth 1 -type f -exec rm -rf {} \;

In [ ]:
!cd .. 
!rm -rf cfg/ 
!mkdir cfg 

In [76]:
!echo -e 'Humans' > data/obj.names
!echo -e 'classes = 1\ntrain  = /content/dataset/train.txt\nvalid  = /content/dataset/valid.txt\nnames = /content/darknet/data/obj.names\nbackup = /content/darknet/backup' > data/obj.data

In [ ]:
with open('/content/dataset/train.txt', 'w') as file_train:
  for train_img in train_images:
    file_train.write

file_val = open('/content/dataset/val.txt', 'w')
file_test = open('/content/dataset/test.txt', 'w')
